In [ ]:
import itertools
import json
import os
from datetime import datetime

from src.universe_utils import load_all_universes  # new helper
from src.pipeline_factory import make_train_test_matrices
from src.model_zoo import choose_model
from sklearn.pipeline import Pipeline